# Composability

This notebook shows off some basic functionality around LangChain Expression Language, which makes it really easy to compose arbitrary chains.

For a much deeper dive, see:

- [Full LangChain Expression Language Documentation](https://js.langchain.com/docs/expression_language/)

## Basic Composability

In [1]:
import { ChatOpenAI } from "@langchain/openai";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { StringOutputParser } from "@langchain/core/output_parsers";


In [2]:
const prompt = ChatPromptTemplate.fromTemplate("Tell me a joke about {topic}");
const model = new ChatOpenAI({
  modelName: "gpt-3.5-turbo",
  temperature: 0,
});
const outputParser = new StringOutputParser();

In [3]:
const chain = prompt.pipe(model).pipe(outputParser);

In [4]:
await chain.invoke({ topic: "bears "});

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

### Batching

In [5]:
await chain.batch([{ topic: "bears "}, { topic: "clowns" }]);

[
  "Why don't bears wear shoes?\n\nBecause they have bear feet!",
  "Why don't clowns ever use an elevator?\n" +
    "\n" +
    `Because they're always taking the "funny" way!`
]

### Streaming

In [6]:
for await (const stream of await chain.stream({ topic: "bears" })) {
  console.log(stream);
}


Why
 don
't
 bears
 wear
 shoes
?


Because
 they
 have
 bear
 feet
!



## RunnablePassthrough

In [7]:
import { TavilySearchAPIRetriever } from "@langchain/community/retrievers/tavily_search_api";

const retriever = new TavilySearchAPIRetriever();

const prompt = ChatPromptTemplate.fromTemplate(`Answer the question based only on the context provided:

Context: {context}

Question: {question}`);

In [8]:
const chain = prompt.pipe(model).pipe(outputParser);

In [9]:
const question = "what is langsmith"
const context = "langsmith is a testing and observability platform built by the langchain team"
await chain.invoke({ question, context })

"Langsmith is a testing and observability platform developed by the Langchain team."

In [10]:
import { RunnablePassthrough } from "@langchain/core/runnables";
import { formatDocumentsAsString } from "langchain/util/document";

const retrievalChain = RunnablePassthrough.assign({
  context: async (i: { question: string }) =>
    formatDocumentsAsString(await retriever.getRelevantDocuments(i.question))
}).withConfig({ runName: "passthrough" }).pipe(chain);

In [11]:
await retrievalChain.invoke({ question })

"LangSmith is a platform that helps trace and evaluate language model applications and intelligent ag"... 354 more characters

## RunnableParallel

In [12]:
import { RunnableParallel } from "@langchain/core/runnables";

In [13]:
const prompt = ChatPromptTemplate.fromTemplate("{question}");
const simpleChain = prompt.pipe(model).pipe(outputParser);

In [14]:
const parallelChain = new RunnableParallel({
  steps: {
    retrievedAnswer: retrievalChain,
    simpleAnswer: simpleChain
  }
});

In [15]:
await parallelChain.invoke({ question });

{
  simpleAnswer: `I'm sorry, but I couldn't find any information about "langsmith." It is possible that it may be a mi`... 97 more characters,
  retrievedAnswer: "LangSmith is a platform or tool that helps developers track the performance, debug issues, and evalu"... 446 more characters
}

In [16]:
for await (const stream of await parallelChain.stream({ question })) {
  console.log(stream);
}

{ simpleAnswer: "" }
{ simpleAnswer: "I" }
{ simpleAnswer: "'m" }
{ simpleAnswer: " sorry" }
{ simpleAnswer: "," }
{ simpleAnswer: " but" }
{ simpleAnswer: " I" }
{ simpleAnswer: " couldn" }
{ simpleAnswer: "'t" }
{ simpleAnswer: " find" }
{ simpleAnswer: " any" }
{ simpleAnswer: " information" }
{ simpleAnswer: " about" }
{ simpleAnswer: ' "' }
{ simpleAnswer: "lang" }
{ simpleAnswer: "smith" }
{ simpleAnswer: '."' }
{ simpleAnswer: " It" }
{ simpleAnswer: " is" }
{ simpleAnswer: " possible" }
{ simpleAnswer: " that" }
{ simpleAnswer: " it" }
{ simpleAnswer: " may" }
{ simpleAnswer: " be" }
{ simpleAnswer: " a" }
{ simpleAnswer: " miss" }
{ simpleAnswer: "p" }
{ simpleAnswer: "elling" }
{ simpleAnswer: " or" }
{ simpleAnswer: " a" }
{ simpleAnswer: " lesser" }
{ simpleAnswer: "-known" }
{ simpleAnswer: " term" }
{ simpleAnswer: "." }
{ simpleAnswer: " Could" }
{ simpleAnswer: " you" }
{ simpleAnswer: " please" }
{ simpleAnswer: " provide" }
{ simpleAnswer: " more" }
{ simpleAnswer: " 